In [2]:
from sqlalchemy import Table, Row, Column, create_engine, MetaData, Integer, String, ForeignKey, Date, select
from sqlalchemy.orm import DeclarativeBase
import openpyxl
import numpy as np

In [3]:
import os
engine = create_engine("sqlite:///mortality.db", echo=True, pool_pre_ping=True)

assert os.path.isfile("mortality.db")

class Base(DeclarativeBase):
    pass

In [4]:
engine

Engine(sqlite:///mortality.db)

In [5]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

In [ ]:
class Country(Base):
    __tablename__ = "countrys"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(255))
    code: Mapped[str] = mapped_column(String(255))

    def __repr__(self) -> str:
        return f"Country(id={self.id!r}, name={self.name!r}, code={self.code!r})"
    

# groups now needs to be normalised
class AgeGroup(Base):
    __tablename__ = "age_groups"

    id: Mapped[int] = mapped_column(primary_key=True)
    age: Mapped[str] = mapped_column(String(255))

    def __repr__(self):
        return f""
    

class Deaths(Base):
    __tablename__ = "deaths"

    id: Mapped[int] = mapped_column(primary_key=True)
    country_id: Mapped[int] = mapped_column(ForeignKey("countrys.id"))
    year: Mapped[Date] = mapped_column(Date)
    age_group_id: Mapped[str] = mapped_column(ForeignKey("age_groups.id"))
    sex: Mapped[str] = mapped_column(String(255))
    number_of_deaths: Mapped[int] = mapped_column()
    death_rate_per_100000: Mapped[int] = mapped_column()

    def __repr__(self):
        return f"Deaths(id={self.id!r}, country_id={self.country_id!r}, year={self.year!r}, age_group={self.age_group!r}, sex={self.sex!r}, deaths_per_100k={self.deaths_per_100k!r})"



In [7]:
Base.metadata

MetaData()

In [8]:
Base.registry

In [9]:
Base.metadata.create_all(engine)

2025-04-05 15:19:20,280 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-05 15:19:20,283 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("countrys")
2025-04-05 15:19:20,285 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-05 15:19:20,288 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("deaths")
2025-04-05 15:19:20,289 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-05 15:19:20,292 INFO sqlalchemy.engine.Engine COMMIT


In [10]:
import pandas as pd

filename = "IHME_GBD_2010_MORTALITY_AGE_SPECIFIC_BY_COUNTRY_1970_2010.csv"

df = pd.read_csv(filename).rename(columns={
    "Country Code" : "code",
    "Country Name" : "name",
    "Year": "year",
    "Age Group" : "age_group",
    "Sex" : "sex",
    "Number of Deaths" : "number_of_deaths",
    "Death Rate Per 100,000" : "death_rate_per_100000"
})

countrys = pd.DataFrame(df[["name", "code"]].drop_duplicates().reset_index(drop=True))




                     name code
0             Afghanistan  AFG
1                  Angola  AGO
2                 Albania  ALB
3                 Andorra  AND
4    United Arab Emirates  ARE
..                    ...  ...
182                 Samoa  WSM
183                 Yemen  YEM
184          South Africa  ZAF
185                Zambia  ZMB
186              Zimbabwe  ZWE

[187 rows x 2 columns]


In [11]:
# countrys.to_sql(name="countrys", if_exists="append", con=engine, index=True, index_label="id")

In [12]:
deaths = pd.DataFrame(df)

In [22]:
print(deaths[["age_group", "code"]])
deaths.drop(columns=["code", "name"])
deaths.combine

       age_group code
0       0-6 days  AFG
1       0-6 days  AFG
2       0-6 days  AFG
3      7-27 days  AFG
4      7-27 days  AFG
...          ...  ...
58900  80+ years  ZWE
58901  80+ years  ZWE
58902   All ages  ZWE
58903   All ages  ZWE
58904   All ages  ZWE

[58905 rows x 2 columns]


,year,age_group,sex,number_of_deaths,death_rate_per_100000
0,1970,0-6 days,Male,"19,241","318,292.90"
1,1970,0-6 days,Female,"12,600","219,544.20"
2,1970,0-6 days,Both,"31,840","270,200.70"
3,1970,7-27 days,Male,"15,939","92,701.00"
4,1970,7-27 days,Female,"11,287","68,594.50"
...,...,...,...,...,...
58900,2010,80+ years,Female,"7,606","16,179.40"
58901,2010,80+ years,Both,"13,305","16,633.80"
58902,2010,All ages,Male,"82,010","1,320.10"
58903,2010,All ages,Female,"77,420","1,211.20"


In [61]:
from sqlalchemy.orm import Session

# Insert deaths into deaths table. Code and name need to be replaced with country.id
# For each row, the country needs to mapped to its id.

# create dict(hashtable of counry names and ids)
# want to return each result row as class

countrys_dict = {}

def hash(str: str):
    return countrys_dict[str]

with Session(engine) as session:
    for country in session.scalars(select(Country)):
        countrys_dict[country.name] = country.id

deaths["country_id"] = deaths["name"].apply(lambda str: countrys_dict[str])
deaths = deaths.drop(columns=["code", "name"])


2025-04-05 17:40:33,331 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-05 17:40:33,333 INFO sqlalchemy.engine.Engine SELECT countrys.id, countrys.name, countrys.code 
FROM countrys
2025-04-05 17:40:33,333 INFO sqlalchemy.engine.Engine [cached since 6528s ago] ()
2025-04-05 17:40:33,343 INFO sqlalchemy.engine.Engine ROLLBACK
       year  age_group     sex number_of_deaths death_rate_per_100000  \
0      1970   0-6 days    Male           19,241            318,292.90   
1      1970   0-6 days  Female           12,600            219,544.20   
2      1970   0-6 days    Both           31,840            270,200.70   
3      1970  7-27 days    Male           15,939             92,701.00   
4      1970  7-27 days  Female           11,287             68,594.50   
...     ...        ...     ...              ...                   ...   
58900  2010  80+ years  Female            7,606             16,179.40   
58901  2010  80+ years    Both           13,305             16,633.80   
58902  20